In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."
assert TAVILY_API_KEY, "TAVILY_API_KEY is missing. Check your .env file."

print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")
print("Tavily Key loaded:", TAVILY_API_KEY[:6] + "...")

# Initialize LLMs
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# Prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a company that makes {product}?",
)


# Example: Use OpenAI
chain = prompt | openai_llm
response = chain.invoke({"product": "smart home devices"})
print("OpenAI response:", response)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
Tavily Key loaded: tvly-d...
OpenAI response: content='"IntelliHome Innovations"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 21, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cqh4iDLliWN7OleUUTpMFgA4kSwq9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5604-26ab-7460-9b9f-36f90edaea71-0' usage_metadata={'input_tokens': 21, 'output_tokens': 7, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-4o-mini")

# Agent A: extractor
prompt_a = ChatPromptTemplate.from_messages([
    ("system", "Extract key facts as bullet points. No extra talk."),
    MessagesPlaceholder("messages"),
])
agent_a = prompt_a | llm   # chain, not create_agent

# Agent B: writer
prompt_b = ChatPromptTemplate.from_messages([
    ("system", "Write a clean final answer using ONLY the facts provided."),
    MessagesPlaceholder("messages"),
])
agent_b = prompt_b | llm   # chain, not create_agent

user_text = "We shipped v1. Latency is high. Users want filters. ETA is Jan 10."

# 1) Call Agent A
a_result = agent_a.invoke({"messages": [{"role": "user", "content": user_text}]})
facts = a_result.content

# 2) Pass Agent A output to Agent B
b_result = agent_b.invoke({
    "messages": [{"role": "user", "content": f"FACTS:\n{facts}"}]
})

print("Agent A output:\n", facts)
print("\nAgent B output:\n", b_result.content)

Agent A output:
 - Version 1 (v1) has been shipped.
- Current latency is high.
- Users are requesting filters.
- Estimated Time of Arrival (ETA) for improvements is January 10.

Agent B output:
 Version 1 (v1) has been shipped, but currently, latency is high and users are requesting filters. Improvements are expected to be implemented by January 10.
